In [1]:
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np

In [2]:
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [14]:
url_head = 'https://arxiv.org'

In [3]:
r = requests.get('https://arxiv.org/list/cs/recent?show=100', headers=headers)

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')
soup

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Computer Science  authors/titles recent submissions</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="//static.arxiv.org/static/browse/0.3.2.8/css/arXiv.css?v=20220215" media="screen" rel="stylesheet" type="text/css"/>
<link href="/bibex/bibex.css?20181009" media="screen" rel="stylesheet" type="text/css"/>
<link href="https://static.arxiv.org/static/browse/0.3.8/css/browse_search.css" media="screen" rel="stylesheet" type="text/css">
<link href="http://arxiv.org/rss/cs" rel="alternate" title="Computer Science " type="application/rss+xml"/>
<script src="/js/mathjaxToggle.min.js" type="text/javascript"></script>
</link></head>
<body class="with-cu-identity">
<div id="cu-identity">
<div id="cu-logo">
<a href="https://www.corne

In [15]:
list_url = soup.select('div[id="dlpage"] a[title="Abstract"]')
list_url[0]['href']

'/abs/2403.00762'

In [40]:
len(list_url)

100

In [41]:
list_title = []
list_subject = []
list_author = []
list_abstract = []

for url_tail in list_url:
    url_complete = url_head + url_tail['href']
    one = requests.get(url_complete,headers)
    one_soup = BeautifulSoup(one.text, 'html.parser')

    title = one_soup.select('div[id="abs"] h1[class="title mathjax"]')
    list_title.append(title[0].text.strip().replace('Title:', ''))

    subjects = one_soup.select('div[id="abs"] td[class="tablecell subjects"]')
    list_subject.append(subjects[0].text.replace('\n', ''))

    authors = one_soup.select('div[id="abs"] div[class="authors"] a')
    authors_names = [author.text for author in authors]
    authors_string = '; '.join(authors_names)
    list_author.append(authors_string)

    abstract = one_soup.select('div[id="abs"] blockquote[class="abstract mathjax"]')
    list_abstract.append(abstract[0].text.strip().replace('Abstract:', ''))

    sleep(2) 


In [42]:
out_df = pd.DataFrame({'title': list_title, 'subjects': list_subject, 'author': list_author, 'abstract': list_abstract})
out_df

,title,subjects,author,abstract
0,Point Could Mamba: Point Cloud Learning via St...,Computer Vision and Pattern Recognition (cs.CV),Tao Zhang; Xiangtai Li; Haobo Yuan; Shunping J...,"In this work, for the first time, we demonstra..."
1,Mitigating Reversal Curse via Semantic-aware P...,Computation and Language (cs.CL); Artificial I...,Qingyan Guo; Rui Wang; Junliang Guo; Xu Tan; J...,While large language models (LLMs) have achiev...
2,An Experimental Study of Low-Latency Video Str...,Multimedia (cs.MM); Performance (cs.PF),Imran Khan; Tuyen X. Tran; Matti Hiltunen; The...,Low-latency video streaming over 5G has become...
3,AtP*: An efficient and scalable method for loc...,Machine Learning (cs.LG); Computation and Lang...,János Kramár; Tom Lieberum; Rohin Shah; Neel N...,Activation Patching is a method of directly co...
4,Neural Acceleration of Incomplete Cholesky Pre...,"Distributed, Parallel, and Cluster Computing (...",Joshua Dennis Booth; Hongyang Sun; Trevor Garnett,The solution of a sparse system of linear equa...
...,...,...,...,...
95,Do Zombies Understand? A Choose-Your-Own-Adven...,Computation and Language (cs.CL),Ariel Goldstein; Gabriel Stanovsky,Recent advances in LLMs have sparked a debate ...
96,"Graph Homomorphism, Monotone Classes and Bound...",Computational Complexity (cs.CC); Logic in Com...,Tala Eagling-Vose; Barnaby Martin; Daniel Paul...,A recent paper describes a framework for study...
97,Analyzing Divergence for Nondeterministic Prob...,Logic in Computer Science (cs.LO),Hao Wu; Yuxi Fu; Huan Long; Xian Xu; Wenbo Zhang,Branching and weak probabilistic bisimilaritie...
98,Multiple Ways of Working with Users to Develop...,Human-Computer Interaction (cs.HC); Robotics (...,Amal Nanavati; Max Pascher; Vinitha Ranganeni;...,Despite the growth of physically assistive rob...


In [45]:
out_df.to_csv('./output/data.csv', index=False)